In [1]:
from bs4 import *
import pandas as pd

from selenium import webdriver

from urllib.parse import urljoin
from time import sleep

In [2]:
def get_business_range(url, driver):
    driver.get(url)
    
    sleep(0.5)
    html_text = driver.page_source
    soup = BeautifulSoup(html_text, 'html.parser')

    business_list = soup.find_all('td', id='jbqk-businessRange')[0].text.split(',')
    
    return [b.strip() for b in business_list]

In [3]:


url = 'http://www.cfachina.org/informationpublicity/futurescompanyinformantionpublicity/qhgsjbqk/qhgsxxgsxqy/?id=0216&organId=G01003&index=0'

driver = webdriver.Firefox()

print(get_business_range(url, driver))

driver.quit()

['商品期货经纪', '资产管理业务资格', '投资咨询业务资格', '金融期货经纪']


In [4]:
class FutureCompanyInfo:
    def __init__(self, name, url):
        self.Name = name
        self.Url = url
        self.BusinessRange = []
        
    def set_business_range(self, business_range):
        self.BusinessRange = business_range

In [5]:
url = 'http://www.cfachina.org/informationpublicity/futurescompanyinformantionpublicity/qhgsjbqk/'
driver = webdriver.Firefox()
driver.get(url)

sleep(3)

page_num = 1
all_companies_info = []

while(page_num <= 8):
    html_text = driver.page_source
    soup = BeautifulSoup(html_text, 'html.parser')
    
    for c in soup.find_all('td', class_='b-cnt2'):
        link = c.find('a', href=True)['href']
        
        all_companies_info.append(FutureCompanyInfo(c.text.strip(), urljoin(url, link)))
        

    driver.find_element_by_class_name('js-page-next.js-page-action.ui-pager').click()
    
    sleep(2)
    page_num += 1

driver.quit()

print(len(all_companies_info))

150


In [7]:
driver = webdriver.Firefox()

for info in all_companies_info:
    print(get_business_range(info.Url, driver))
    info.set_business_range(get_business_range(info.Url, driver))
    
driver.quit()

['商品期货经纪', '基金销售业务资格', '资产管理业务资格', '投资咨询业务资格', '金融期货经纪']
['商品期货经纪', '资产管理业务资格', '投资咨询业务资格', '金融期货经纪']
['商品期货经纪', '资产管理业务资格', '投资咨询业务资格', '金融期货经纪']
['商品期货经纪', '资产管理业务资格', '投资咨询业务资格', '金融期货经纪']
['商品期货经纪', '资产管理业务资格', '投资咨询业务资格', '金融期货经纪']
['资产管理业务资格', '商品期货经纪', '金融期货经纪', '投资咨询业务资格']
['商品期货经纪', '基金销售业务资格', '资产管理业务资格', '投资咨询业务资格', '金融期货经纪']
['投资咨询业务资格', '资产管理业务资格', '商品期货经纪', '金融期货经纪']
['商品期货经纪', '资产管理业务资格', '投资咨询业务资格', '金融期货经纪']
['商品期货经纪', '资产管理业务资格', '投资咨询业务资格', '金融期货经纪']
['资产管理业务资格', '金融期货经纪', '商品期货经纪']
['金融期权', '商品期货经纪', '金融期货经纪', '投资咨询业务资格', '资产管理业务资格', '基金销售业务资格', '商品期权']
['资产管理业务资格', '金融期货经纪', '投资咨询业务资格', '商品期货经纪']
['投资咨询业务资格', '商品期货经纪', '金融期货经纪', '资产管理业务资格']
['商品期货经纪', '资产管理业务资格', '投资咨询业务资格', '金融期货经纪']
['商品期货经纪', '资产管理业务资格', '投资咨询业务资格', '金融期货经纪']
['商品期货经纪', '资产管理业务资格', '投资咨询业务资格', '金融期货经纪']
['商品期货经纪', '金融期货经纪']
['商品期货经纪', '投资咨询业务资格', '金融期货经纪']
['商品期货经纪', '金融期货经纪']
['资产管理业务资格', '金融期货经纪', '商品期货经纪', '投资咨询业务资格']
['资产管理业务资格', '商品期货经纪', '金融期货经纪', '投资咨询业务资格', '基金销售业务资格']
['资产管理业务资格', '金融期货

In [8]:
business_range_set = set()

for info in all_companies_info:
    business_range_set |= set(info.BusinessRange)
    
business_range_set

{'商品期权',
 '商品期货经纪',
 '基金销售业务资格',
 '投资咨询业务资格',
 '股票期权经纪',
 '股票现货经纪',
 '资产管理业务资格',
 '金融期权',
 '金融期货经纪'}

In [9]:
business_range_list = sorted(list(business_range_set))

df = pd.DataFrame(columns=business_range_list)
df.index.name = '机构名称'

df

,商品期权,商品期货经纪,基金销售业务资格,投资咨询业务资格,股票期权经纪,股票现货经纪,资产管理业务资格,金融期权,金融期货经纪
机构名称,,,,,,,,,


In [13]:
def get_bool_list_from_business_range(business_range):
    return [biz in business_range for biz in business_range_list]


for info in all_companies_info:
    df.loc[info.Name] = get_bool_list_from_business_range(info.BusinessRange)

df

,商品期权,商品期货经纪,基金销售业务资格,投资咨询业务资格,股票期权经纪,股票现货经纪,资产管理业务资格,金融期权,金融期货经纪
机构名称,,,,,,,,,
方正中期期货有限公司,False,True,True,True,False,False,True,False,True
金鹏期货经纪有限公司,False,True,False,True,False,False,True,False,True
国都期货有限公司,False,True,False,True,False,False,True,False,True
中粮期货有限公司,False,True,False,True,False,False,True,False,True
北京首创期货有限责任公司,False,True,False,True,False,False,True,False,True
...,...,...,...,...,...,...,...,...,...
中金期货有限公司,False,True,False,False,False,False,True,False,True
东方汇金期货有限公司,False,True,False,False,False,False,True,False,True
财信期货有限公司,False,True,False,True,False,False,True,False,True


In [12]:
df.to_excel('./ChineseFuturesCompaniesBusinessRange.xls')

/var/folders/x5/cxp2ts2n7h93w5g276jhwc_00000gn/T/ipykernel_96606/234761747.py:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  df.to_excel('./ChineseFuturesCompaniesBusinessRange.xls')
